## Loan Data Cleaning Pipeline
Complete Data Cleaning Workflow

## Step 1: Import Libraries

In [6]:
import pandas as pd
import numpy as np

print("Libraries Imported Successfully ✅")

ModuleNotFoundError: No module named 'pandas'

## 🟢 Step 2: Load Dataset

If your file is uploaded manually in Colab:

In [ ]:
# Upload file manually in Colab first (if needed)
# from google.colab import files
# uploaded = files.upload()

# Load dataset
df = pd.read_csv('loan_prediction.csv')

df.head()

## 🟢 Step 3: Understand the Data

In [ ]:
# Shape of dataset
print("Dataset Shape:")
print(f"Rows: {df.shape[0]}, Columns: {df.shape[1]}")
print()

# Info
print("Dataset Info:")
df.info()
print()

# Summary Statistics
print("Summary Statistics:")
print(df.describe(include='all'))

## 🟢 Step 4: Check Missing Values

In [ ]:
# Missing values summary
missing = df.isnull().sum()
missing = missing[missing > 0].sort_values(ascending=False)

print("Missing Values:")
print(missing)

## 🟢 Step 5: Handle Missing Values (Improved Way)

⚠️ We will NOT use `inplace=True`  
⚠️ We will NOT modify original dataframe

In [ ]:
# Create copy
df_clean = df.copy()

# Separate columns
cat_cols = df_clean.select_dtypes(include='object').columns
num_cols = df_clean.select_dtypes(include=['int64', 'float64']).columns

# Fill categorical columns with mode
for col in cat_cols:
    if df_clean[col].isnull().sum() > 0:
        mode_value = df_clean[col].mode()[0]
        df_clean[col] = df_clean[col].fillna(mode_value)

# Fill numerical columns with median
for col in num_cols:
    if df_clean[col].isnull().sum() > 0:
        median_value = df_clean[col].median()
        df_clean[col] = df_clean[col].fillna(median_value)

print("Missing Values Handled ✅")

## 🟢 Step 6: Fix Special Column (Dependents)

Dependents column contains "3+" → we convert it to 3

In [ ]:
# Replace 3+ with 3
df_clean['Dependents'] = df_clean['Dependents'].replace('3+', '3')

# Convert to numeric
df_clean['Dependents'] = pd.to_numeric(df_clean['Dependents'])

## 🟢 Step 7: Standardize Text Columns

In [ ]:
for col in cat_cols:
    df_clean[col] = df_clean[col].str.strip().str.lower()

print("Text Standardized ✅")

## 🟢 Step 8: Convert Categorical Columns to Category Type

In [ ]:
for col in cat_cols:
    df_clean[col] = df_clean[col].astype('category')

print("Data Types Fixed ✅")

## 🟢 Step 9: Handle Outliers (Capping at 99%)

In [ ]:
outlier_cols = ['LoanAmount', 'ApplicantIncome']

for col in outlier_cols:
    if col in df_clean.columns:
        upper = df_clean[col].quantile(0.99)
        df_clean[col] = np.where(df_clean[col] > upper, upper, df_clean[col])

print("Outliers Capped ✅")

## 🟢 Step 10: Drop Unnecessary Column

Loan_ID is not useful for ML.

In [ ]:
df_clean = df_clean.drop(columns=['Loan_ID'])

print("Loan_ID Removed ✅")

## 🟢 Step 11: Final Clean Dataset Check

In [ ]:
print("Final Dataset Info:")
df_clean.info()

print()
print("Remaining Missing Values:")
print(df_clean.isnull().sum())

## 🟢 Step 12: Save Clean Dataset

In [ ]:
df_clean.to_csv('cleaned_loan_data.csv', index=False)

print("Cleaned Dataset Saved Successfully 🎉")

## 🔥 BONUS: Final Reusable Pipeline Function

If you want everything inside ONE function:

In [ ]:
def clean_loan_data(df):
    """
    Reusable function to clean loan prediction dataset
    """
    df = df.copy()

    # Separate columns
    cat_cols = df.select_dtypes(include='object').columns
    num_cols = df.select_dtypes(include=['int64', 'float64']).columns

    # Handle missing values
    for col in cat_cols:
        if df[col].isnull().sum() > 0:
            df[col] = df[col].fillna(df[col].mode()[0])

    for col in num_cols:
        if df[col].isnull().sum() > 0:
            df[col] = df[col].fillna(df[col].median())

    # Fix Dependents column
    if 'Dependents' in df.columns:
        df['Dependents'] = df['Dependents'].replace('3+', '3')
        df['Dependents'] = pd.to_numeric(df['Dependents'])

    # Standardize text
    for col in cat_cols:
        df[col] = df[col].str.strip().str.lower()

    # Convert to category
    for col in cat_cols:
        df[col] = df[col].astype('category')

    # Outlier handling
    for col in ['LoanAmount', 'ApplicantIncome']:
        if col in df.columns:
            upper = df[col].quantile(0.99)
            df[col] = np.where(df[col] > upper, upper, df[col])

    # Drop Loan_ID
    if 'Loan_ID' in df.columns:
        df = df.drop(columns=['Loan_ID'])

    return df

### Usage Example

In [ ]:
# Load original dataset
df = pd.read_csv('loan_prediction.csv')

# Apply cleaning function
df_clean = clean_loan_data(df)

# Verify results
print("Cleaning completed!")
print(df_clean.head())